In [1]:
from selenium import webdriver
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from scrapy.http import TextResponse

In [14]:
%%time
title_ls = []
clicked_ls = []
time_ls = []
link_ls = []

for page in range(1,31+1):
    if page % 100 ==0 : print('{} page is over.'.format(page))
    url = 'http://www.chaintalk.io/archive/talkbox/p{}'.format(page)
    req = requests.get(url)
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    for content in range(1,20+1):
        try : 
            if content % 10 == 0 : time.sleep(1)
            testing_ls = response.xpath('//*[@id="fboardlist"]/div/table/tbody/tr[{}]/td[2]/a/text()'.format(content)).extract()[0]
            testing_ls[1:].replace(' ','')
            title_ls.append(testing_ls)
            clicked_ls.append(response.xpath('//*[@id="fboardlist"]/div/table/tbody/tr[{}]/td[5]/text()'.format(content)).extract()[0])
            time_ls.append(response.xpath('//*[@id="fboardlist"]/div/table/tbody/tr[{}]/td[4]/text()'.format(content)).extract()[0])
            link_ls.append(response.xpath('//*[@id="fboardlist"]/div/table/tbody/tr[{}]/td[2]/a/@href'.format(content)).extract()[0])
        except : print('pass')

CPU times: user 3.06 s, sys: 15.1 ms, total: 3.07 s
Wall time: 1min 10s


In [15]:
df = pd.DataFrame()

df['title'] = title_ls
df['time'] = time_ls
df['click'] = clicked_ls
df['link'] = link_ls

In [16]:
df.tail()

,title,time,click,link
615,\n 반갑습니다^^,03-07,1058,http://www.chaintalk.io/archive/talkbox/90?pag...
616,\n 가입인사,03-07,950,http://www.chaintalk.io/archive/talkbox/89?pag...
617,\n 가입인사드립니다. ...,03-07,1112,http://www.chaintalk.io/archive/talkbox/86?pag...
618,\n 가입인사드리며..양큰님 따라 가렵니다. ...,03-06,996,http://www.chaintalk.io/archive/talkbox/75?pag...
619,\n 방갑습니다..^^* ...,03-06,1389,http://www.chaintalk.io/archive/talkbox/68?pag...


In [27]:
%%time
content_ls = []
for idx in range(len(df)):
    if idx % 10 ==0 : print(idx)
    req = requests.get(df['link'].values[idx])
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    process_ls = ','.join(response.xpath('//*[@id="bo_v_con"]/div/p/text()').extract()).replace('\xa0','').replace(',','')
    content_ls.append(process_ls)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
CPU times: user 14.2 s, sys: 315 ms, total: 14.6 s
Wall time: 3min 30s


In [28]:
df1 = pd.DataFrame()

df1['title'] = title_ls
df1['time'] = time_ls
df1['click'] = clicked_ls
df1['content'] = content_ls

In [29]:
df1.tail()

,title,time,click,content
615,\n 반갑습니다^^,03-07,1058,깊이 아는 분야는 아니지만 많이 배우도록 하겠습니다.
616,\n 가입인사,03-07,950,안녕하세요.땡글의 homeless 딩가딩입니다.블럭체인에 대한 기술적 지식은 희박하...
617,\n 가입인사드립니다. ...,03-07,1112,가입인사드립니다.많은 고급정보을 접할수있는 계기가 될것 같습니다.감사합니다.
618,\n 가입인사드리며..양큰님 따라 가렵니다. ...,03-06,996,땡글에서 양큰님께 반하여 여기까지오게되었습니다. 우리 2세들의 세상을 저희가 멋지게...
619,\n 방갑습니다..^^* ...,03-06,1389,반갑습니다..번창하세요..근데 블럭체인이 뭔뜻??그리고 아이디중복체크 없어요..ㅠㅠ.


In [31]:
df1.to_csv('chaintalk_data.csv',index=False)